In [3]:
from utils import common

ModuleNotFoundError: No module named 'utils'

In [ ]:
#Create S3 client for Supabase 
s3 = common.get_s3_client() 

#Example: list files in "raw/" folder 
response = s3.list_objects_v2(Bucket=bucket_name, Prefix="raw/") 
for obj in response.get("Contents", []): 
    print(obj["Key"]) 


raw/!!DO_NOT_DELETE!!.txt
raw/Evening_Ride.fit
raw/Evening_Ride.tcx
raw/Solo_Ride.fit


In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
from fitparse import FitFile
import numpy as np
import os
from io import BytesIO  # Needed for FIT parsing

def get_activity_date(file_bytes, extension): 
    if extension == ".fit":
        try:
            fitfile = FitFile(BytesIO(file_bytes))  
            for record in fitfile.get_messages('record'):
                for field in record:
                    if field.name == "timestamp":
                        return field.value.date().isoformat()
        except Exception as e:
            print(f"Failed to parse .fit file: {e}") 
    elif extension == ".tcx":
        try:
            tree = ET.ElementTree(ET.fromstring(file_bytes.decode("utf-8")))
            namespaces = {'tcx': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2'}
            time_elem = tree.find('.//tcx:Trackpoint/tcx:Time', namespaces)
            if time_elem is not None:
                return pd.to_datetime(time_elem.text).date().isoformat()
        except Exception as e:
            print(f"Failed to parse .tcx file: {e}")
    return "unknown"


In [4]:
import uuid

response = s3.list_objects_v2(Bucket=bucket_name, Prefix="raw/")
for obj in response.get("Contents", []):
    raw_key = obj["Key"]
    if not raw_key.endswith((".fit", ".tcx")):
        continue

    # Download raw bytes only for parsing (safe, read-only)
    raw_obj = s3.get_object(Bucket=bucket_name, Key=raw_key)
    file_bytes = raw_obj["Body"].read()
    extension = os.path.splitext(raw_key)[-1]

    # Extract activity date
    activity_date = get_activity_date(file_bytes, extension)
    if activity_date == "unknown":
        print(f"Could not extract date from {raw_key}")
        continue

    short_id = str(uuid.uuid4())[:8]
    new_key = f"bronze/original/bronze_activity_{activity_date}_{short_id}{extension}"

    # Safe move: Copy then delete
    s3.copy_object(
        Bucket=bucket_name,
        CopySource={'Bucket': bucket_name, 'Key': raw_key},
        Key=new_key
    )
    s3.delete_object(Bucket=bucket_name, Key=raw_key)

    print(f"Moved: {raw_key} ➜ {new_key}")


Moved: raw/Evening_Ride.fit ➜ bronze/original/bronze_activity_2025-04-14_6a1e0cfe.fit
Moved: raw/Evening_Ride.tcx ➜ bronze/original/bronze_activity_2025-04-14_493c8809.tcx
Moved: raw/Solo_Ride.fit ➜ bronze/original/bronze_activity_2025-04-26_1f6a6649.fit
